# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings
warnings.filterwarnings('ignore')

# Import API key
from api_keys import geoapify_key

In [18]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,udachny,66.4167,112.4000,14.87,54,16,3.49,RU,1722797922
1,1,port-aux-francais,-49.3500,70.2167,1.85,67,31,10.04,TF,1722797924
2,2,iqaluit,63.7506,-68.5145,5.85,93,100,2.06,CA,1722797926
3,3,san luis de la loma,17.2706,-100.8939,30.89,62,75,4.11,MX,1722797927
4,4,sisimiut,66.9395,-53.6735,8.75,76,77,3.70,GL,1722797928


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
map1 = city_data_df.hvplot.points('Lng','Lat',geo = True, tiles = 'OSM', size='Humidity', color = 'City',width=1200, height=700)

# Display the map
map1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [21]:
# Narrow down cities that fit criteria and drop any results with null values
idealCity = city_data_df.loc[(city_data_df['Max Temp'] > 21)& (city_data_df['Max Temp'] < 31) & (city_data_df['Humidity'] < 30),:]

# Drop any rows with null values
idealCity = idealCity.dropna(how = 'any')

# Display sample data
idealCity

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,26,al jawf,29.5000,38.7500,30.21,16,5,3.52,SA,1722797955
62,62,al bawiti,28.3492,28.8659,30.78,20,0,3.68,EG,1722797998
167,167,kananga,-5.8958,22.4178,25.87,24,4,1.60,CD,1722798133
248,248,buka,40.8108,69.1986,24.98,22,53,0.23,UZ,1722798233
295,295,tinogasta,-28.0632,-67.5649,22.03,16,0,4.58,AR,1722798290
305,305,ain oussera,35.4514,2.9058,30.39,20,0,5.33,DZ,1722798302
323,323,el idrissia,34.4454,2.5275,28.33,14,0,4.34,DZ,1722798324
384,384,chu,43.5983,73.7614,26.37,23,0,0.76,KZ,1722798401
476,476,macia,-25.0269,33.0989,23.18,24,5,3.37,MZ,1722798516
477,477,smithers,54.7804,-127.1743,25.95,28,78,1.19,CA,1722798517


### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idealCity.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
26,26,al jawf,29.5000,38.7500,30.21,16,5,3.52,SA,1722797955,
62,62,al bawiti,28.3492,28.8659,30.78,20,0,3.68,EG,1722797998,
167,167,kananga,-5.8958,22.4178,25.87,24,4,1.60,CD,1722798133,
248,248,buka,40.8108,69.1986,24.98,22,53,0.23,UZ,1722798233,
295,295,tinogasta,-28.0632,-67.5649,22.03,16,0,4.58,AR,1722798290,
305,305,ain oussera,35.4514,2.9058,30.39,20,0,5.33,DZ,1722798302,
323,323,el idrissia,34.4454,2.5275,28.33,14,0,4.34,DZ,1722798324,
384,384,chu,43.5983,73.7614,26.37,23,0,0.76,KZ,1722798401,
476,476,macia,-25.0269,33.0989,23.18,24,5,3.37,MZ,1722798516,
477,477,smithers,54.7804,-127.1743,25.95,28,78,1.19,CA,1722798517,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [43]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'limit': 1,
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
al jawf - nearest hotel: No hotel found
al bawiti - nearest hotel: Old Oasis Hotel
kananga - nearest hotel: Hôtel Résidence
buka - nearest hotel: No hotel found
tinogasta - nearest hotel: Hotel de Turismo
ain oussera - nearest hotel: El Mahdi
el idrissia - nearest hotel: No hotel found
chu - nearest hotel: No hotel found
macia - nearest hotel: No hotel found
smithers - nearest hotel: Sunshine Inn Hotel
san juan - nearest hotel: Hotel San Francisco


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
26,26,al jawf,29.5000,38.7500,30.21,16,5,3.52,SA,1722797955,No hotel found
62,62,al bawiti,28.3492,28.8659,30.78,20,0,3.68,EG,1722797998,Old Oasis Hotel
167,167,kananga,-5.8958,22.4178,25.87,24,4,1.60,CD,1722798133,Hôtel Résidence
248,248,buka,40.8108,69.1986,24.98,22,53,0.23,UZ,1722798233,No hotel found
295,295,tinogasta,-28.0632,-67.5649,22.03,16,0,4.58,AR,1722798290,Hotel de Turismo
305,305,ain oussera,35.4514,2.9058,30.39,20,0,5.33,DZ,1722798302,El Mahdi
323,323,el idrissia,34.4454,2.5275,28.33,14,0,4.34,DZ,1722798324,No hotel found
384,384,chu,43.5983,73.7614,26.37,23,0,0.76,KZ,1722798401,No hotel found
476,476,macia,-25.0269,33.0989,23.18,24,5,3.37,MZ,1722798516,No hotel found
477,477,smithers,54.7804,-127.1743,25.95,28,78,1.19,CA,1722798517,Sunshine Inn Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [45]:
%%capture --no-display

# Configure the map plot
map2 = hotel_df.hvplot.points('Lng','Lat',geo = True, tiles = 'OSM', size='Humidity', color = 'City',width=600, height=300,hover_cols=['Lng','Lat','City','Humidity','Hotel Name', 'Country'])

# Display the map
map2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)